In [ ]:
import nltk
from nltk.corpus import indian
from nltk.tag import tnt
import string
import numpy as np

In [ ]:
nltk.download('indian')
nltk.download('punkt')

In [ ]:
tagged_set = 'bangla.pos'
word_set = indian.sents(tagged_set)
count = 0
for sen in word_set:
    count = count + 1
    sen = "".join([" "+i if not i.startswith("'") and i not in string.punctuation else i for i in sen]).strip()
    print (count, sen)
print ('Total sentences in the tagged file are',count)

train_perc = .86

train_rows = int(train_perc*count)
test_rows = train_rows + 1

print ('Sentences to be trained',train_rows, 'Sentences to be tested against',test_rows)

In [ ]:
data = indian.tagged_sents(tagged_set)
train_data = data[:train_rows] 
test_data = data[test_rows:]
pos_tagger = tnt.TnT()
pos_tagger.train(train_data)
pos_tagger.evaluate(test_data)

In [ ]:
def getFeatures(sentence, index):
    token, tag = sentence[index]
    prev_token = ""
    if index > 0:
        prev_token, prev_tag = sentence[index - 1]
    is_number = False
    try:
        if float(token):
            is_number = True
    except:
        pass
    features_dict = {"token": token,
                     'first_word': index == -1
                     , "lower_cased_token": token
                     , "prev_token": prev_token
                     ,'prefix(1)': sentence[index][0]
                     , "is_number": is_number}
    return features_dict

In [ ]:
data

In [ ]:
getFeatures(data[0],1)

In [ ]:
from sklearn.feature_extraction import DictVectorizer
X_features = []
for sentence in data:
    for k in range(len(sentence)):
        X_features.append(getFeatures(sentence, k))

vectoriser = DictVectorizer()
X = vectoriser.fit_transform(X_features)

In [ ]:
X.shape

In [ ]:
Y = []
for sentence in data:
    for k in range(len(sentence)):
        Y.append(sentence[k][1])

In [ ]:
len(Y)

In [ ]:
from sklearn.model_selection import train_test_split
xtrain, xvalid, ytrain, yvalid = train_test_split(X, Y,
                                                  random_state=32,
                                                  test_size=0.25, shuffle=True)

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import cross_val_score
clf = MultinomialNB()
clf.fit(xtrain,ytrain)

scores = cross_val_score(clf, xtrain, ytrain, cv=10)
print('Bengali POS tagging Accuracy: %.3f' % (np.mean(scores)*100))

In [ ]:
from sklearn.metrics import confusion_matrix
predictions = clf.predict_proba(xvalid)
predicted_lables = clf.predict(xvalid)

confusion_matrix = confusion_matrix(yvalid, predicted_lables)

In [ ]:
confusion_matrix